In [ ]:
import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [ ]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [ ]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']

In [ ]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [ ]:
# filters
probChi2_filtered = frame[frame['probChi2'] > 0.3]
error_filtered = probChi2_filtered[probChi2_filtered['x_error'] < 10] # e/sqrt(n) was < 5
error_filtered = error_filtered[error_filtered['y_error'] < 10]
frame = error_filtered[error_filtered['chi2PerDof'] < 1.2]

In [ ]:
frame = frame.sort_values(by='measurementTimestamp')

In [ ]:
# get most frequent mac address
macs = frame['sourceMac'].value_counts()
macs[:5]

In [ ]:
len(macs)

In [ ]:
from math import floor, ceil
import numpy as np
import time

begin = time.time()

width = 80; height = 60; periods = 60 # was 10

bins = np.zeros((periods,height,width))

interval = 120000 # milliseconds
shift = 30000

for k in range(periods):
    #start = min(frame['measurementTimestamp']) + k * interval
    #stop = min(frame['measurementTimestamp']) + (k+1) * interval
    
    start = min(frame['measurementTimestamp']) + k * shift
    stop =  start + interval
    
    timewindow = frame[frame['measurementTimestamp'] < stop]
    timewindow = timewindow[timewindow['measurementTimestamp'] >= start]
    
    #normalization = 0
    
    #bins.append()

    for i in range(len(macs)): # len(macs)
        # filter for mac address
        mac_filtered_frame = timewindow[timewindow['sourceMac'] == macs.index[i]]
        # extract positions
        locations = mac_filtered_frame['coordinates']

        positions1 = pd.DataFrame(locations)
        positions = pd.DataFrame(list(positions1['coordinates']))

        f = lambda x: x/3

        # binned positions, still fractional numbers
        binpos = positions.apply(f)
        
        normalization = 0
        bins_mac = np.zeros((height,width))

        for j in range(len(binpos)):
            if floor(binpos[0][j]) >= -width/2 and floor(binpos[0][j]) < width/2 and \
                floor(binpos[1][j]) >= -height/2 and floor(binpos[1][j]) < height/2:
                bins_mac[height/2 + floor(binpos[1][j])][width/2 + floor(binpos[0][j])] += 1
                normalization += 1
        
        if normalization > 0:
            bins_mac /= normalization
            
        bins[k] += bins_mac
    print(k)
    
end = time.time()
print(end - begin)

In [ ]:
bins[0].sum()

In [ ]:
bins.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(width)
Y = np.arange(height)
Xs, Ys = np.meshgrid(X, Y)

for i in range(periods):
    Z = bins[i]

    ax = Axes3D(fig)
    ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
    ax.set_zlim3d(0, 16)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('surface-%s.png' % number)
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

#im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots(figsize=(12,8))

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

#ax.imshow(im)

for i in range(periods):
    ax.imshow(bins[i], aspect='auto', cmap='hsv', alpha=1)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('map-%s.png' % number)
    
#plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')

x_data, y_data = np.meshgrid( np.arange(width),
                              np.arange(height)*(-1) )

col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
# colors = np.random.choice(col, height*width)

x_data = x_data.flatten()
y_data = y_data.flatten()

for i in range(periods):
    z_data = bins[i].flatten()
    ax.set_zlim3d(0, 16)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('bar-%s.png' % number)

#plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.plot(bins[:,18,60])
plt.show()